In [3]:
import tweepy
import json

In [4]:
# Twitter API credentials
bearer_token = "AAAAAAAAAAAAAAAAAAAAAEeQyQEAAAAAz7RyNP5HHBjnPkdJap%2Fb%2Bna78Dc%3D1czDzE57XZmVW65ezR9d1lGIfIRZQwyQ6EUL0xZYSQy80AZiES"

# Create API client
client = tweepy.Client(bearer_token)

# df = pd.read_csv('data/notes-00000.tsv', sep='\t')
# print(df['tweetId'].head())
#
# tweetid = df['tweetId'][0]
# print(tweetid)
tweetid = '1783159712986382830'

# Get tweet by ID
tweet = client.get_tweet(tweetid, 
                         tweet_fields=['author_id','created_at','public_metrics','source'],
                         user_fields=['affiliation','public_metrics','username','verified','verified_type','parody'])


In [5]:
data = dict(tweet.data)
print(data)

{'text': 'After much deliberation, I voted "no" on the foreign aid supplemental package. \n \nWe had an opportunity to vote for an emergency aid package to Israel six months ago— Senate Democrats blocked this legislation.\n \nThis bill gives money to Gaza, funds NGOs who support the… https://t.co/zP6ARyGXvQ https://t.co/0nTDWoVS36', 'edit_history_tweet_ids': [1783159712986382830], 'author_id': 1074480192, 'public_metrics': {'retweet_count': 1527, 'reply_count': 857, 'like_count': 7020, 'quote_count': 72, 'bookmark_count': 40, 'impression_count': 263048}, 'created_at': datetime.datetime(2024, 4, 24, 15, 42, 59, tzinfo=datetime.timezone.utc), 'id': 1783159712986382830}


In [6]:
def flatten_tweet_data(data):
    flat_data = {}
    for key, value in data.items():
        if isinstance(value, dict):
            flat_data.update(flatten_tweet_data(value))
        else:
            flat_data[key] = value
    return flat_data

In [10]:
flat_data = flatten_tweet_data(data)
# convert datetime to string
# flat_data['created_at'] = flat_data['created_at'].strftime('%Y-%m-%d %H:%M:%S')
print(json.dumps(flat_data, indent=2))

{
  "text": "After much deliberation, I voted \"no\" on the foreign aid supplemental package. \n \nWe had an opportunity to vote for an emergency aid package to Israel six months ago\u2014 Senate Democrats blocked this legislation.\n \nThis bill gives money to Gaza, funds NGOs who support the\u2026 https://t.co/zP6ARyGXvQ https://t.co/0nTDWoVS36",
  "edit_history_tweet_ids": [
    1783159712986382830
  ],
  "author_id": 1074480192,
  "retweet_count": 1527,
  "reply_count": 857,
  "like_count": 7020,
  "quote_count": 72,
  "bookmark_count": 40,
  "impression_count": 263048,
  "created_at": "2024-04-24 15:42:59",
  "id": 1783159712986382830
}
